In [1]:
#
import requests
import pandas as pd
from datetime import datetime, timedelta

# NYC 311 Open Data API 端点
BASE_URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"

# 可选：如果你在 NYC Open Data 申请了 app token，可以填在这里提高配额
APP_TOKEN = None  # 比如 "xxxxxxxxxxxxxxxxxxxxxx"


In [2]:
# 计算最近 10 天的起始时间（按 UTC）
end_dt = datetime.utcnow()
start_dt = end_dt - timedelta(days=10)

# SoQL 需要的时间格式，例如 '2025-02-20T00:00:00'
start_str = start_dt.strftime("%Y-%m-%dT00:00:00")
print("Start time for query (UTC):", start_str)


Start time for query (UTC): 2025-11-28T00:00:00


/tmp/ipython-input-200085760.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_dt = datetime.utcnow()


In [3]:
# 构造请求参数
params = {
    # 只要最近 10 天创建的工单
    "$where": f"created_date >= '{start_str}'",
    # 一次最多拉多少条记录（上限 50000，够你先做 demo 了）
    "$limit": 50000
}

# 如果有 APP_TOKEN 就带上
headers = {}
if APP_TOKEN:
    headers["X-App-Token"] = APP_TOKEN

response = requests.get(BASE_URL, params=params, headers=headers)
response.raise_for_status()  # 如果请求失败会抛异常，方便调试

data = response.json()
print(f"拉取到的记录条数: {len(data)}")


拉取到的记录条数: 50000


In [4]:
# 转成 DataFrame
df = pd.DataFrame(data)

print("列的数量：", df.shape[1])
print("前 5 行示例：")
df.head()


列的数量： 41
前 5 行示例：


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,location,vehicle_type,facility_type,taxi_pick_up_location,bridge_highway_name,bridge_highway_segment,bridge_highway_direction,due_date,road_ramp,taxi_company_borough
0,66968197,2025-11-28T00:00:27.000,2025-11-28T07:02:19.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11220,6814 6 AVENUE,...,"{'latitude': '40.63286039030724', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66973429,2025-11-28T00:00:34.000,2025-11-28T03:36:59.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466,655 EAST 230 STREET,...,"{'latitude': '40.89187241649303', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66979393,2025-11-28T00:00:39.000,2025-12-01T13:17:08.000,HPD,Department of Housing Preservation and Develop...,DOOR/WINDOW,DOOR TO DUMBWAITER,RESIDENTIAL BUILDING,10029,449 EAST 116 STREET,...,"{'latitude': '40.79533814899546', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66973455,2025-11-28T00:00:39.000,2025-11-28T03:37:16.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466,655 EAST 230 STREET,...,"{'latitude': '40.89187241649303', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,66974970,2025-11-28T00:00:42.000,2025-11-28T01:04:52.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11222,574 MANHATTAN AVENUE,...,"{'latitude': '40.722657300952335', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
